In [14]:
import pandas as pd
import math
import glob
import os
import numpy as np
import altair as alt
from read_roi import read_roi_zip
import tifffile.tifffile
from skimage.measure import profile_line
from matplotlib import pyplot as plt
from itertools import groupby
from scipy import ndimage
from scipy.ndimage  import interpolation
import cv2
from skimage.transform import rotate
from scipy.stats import linregress
import imageio
from collections import defaultdict
from scipy.stats import linregress
from scipy.stats import norm
import lmfit
from lmfit import Model, Parameter, report_fit
alt.data_transformers.disable_max_rows()
#vf.enable()
subfactor = 6
searchx = 4
searchy = 2
deltax = searchx * subfactor#search window perpendicular to the axis of comet growth
deltay = searchy * subfactor#search window along the axis of comet growth
pixel_size = 1/9.1287 #um
spf = 5 #s
comet_length = (subfactor * 22) -1 #odd number
to_um = pixel_size / subfactor
comet_length_um = comet_length*pixel_size/subfactor
dir="/Users/ella/Dropbox/000MyProjects/Xenopus/ANALYSIS/Measures/COMETS/25"
def normcomet(array):
    x=(array - np.amin(array))/(np.amax(array) - np.amin(array))
    return(x)
domain = ['Xl','Xb','Xt']
range_ = [ '#1e345c', '#6cacd0', '#e68267']
scalesp2 = alt.Scale(domain = domain, range = range_)

In [ ]:
list_df = []
list_dfunc = []
list_velocity = []
#pix_d={}
empty=0
for root, dirs, files in os.walk(dir):
#assign conditions of assay based on name of folder
#organisation required: in main folder, 1 subfolder/condition, named: SS_tubTTTuM_EBeenM_TXX_YYMMDD_R_AAA, 
#SS: species, TTT concentration tubulin in uM, ee concentration EB1 in nM, XX temperature, YYMMDD date of assay, R # of reaction, AAA # of assay within reaction
#in each subfolder, one .tif file with image sequence and one .zip file with ROIs
    for names in dirs:
        print(names)
        imstack = []
        ip_comet=[]
        dict_var={'specie': names[:2],'tubconc':names[6:9],'ebconc':names[14:17],'temp':names[21:23],'assay':names[24:36]}
        
#extract roi infos from zip file       
        roifile=[f for f in os.listdir(root+"/"+names+"/") if f.endswith('2.zip')]
        rois=read_roi_zip(root+"/"+names+"/"+roifile[0]) 
        roi_values = [ [k,v] for k, v in rois.items() ]
        comet_rois = []
        for i in np.arange(0, len(roi_values)-1, 2):
            comet_rois.append([(roi_values[i][1]['x'][0], roi_values[i][1]['y'][0]), # x,y start point
                               (roi_values[i+1][1]['x'][0], roi_values[i+1][1]['y'][0]), #x,y end point
                               (roi_values[i][1]['position'], roi_values[i+1][1]['position'])]) #starting and ending slice

#open the image sequence        
        im=[f for f in os.listdir(root+"/"+names+"/") if f.endswith('.tif')]
        im = tifffile.imread(root+"/"+names+"/"+im[0])
        im_size = np.shape(im)[1]
        for i in im:
            width = int(i.shape[1] * subfactor)
            height = int(i.shape[0] * subfactor)
            dim = (width, height)
            i = cv2.resize(i, dim, interpolation = cv2.INTER_CUBIC) #split pixels according to subfactor
            imstack.append(i) 
#iterate over comets    
        for i in range(len(comet_rois)):
            total_repcomets = len(comet_rois)-1
            print("Working on assay #" + str(names[24:36]) + ", comet #" + str(i) + "/" + str(total_repcomets) + "       ", end='\r', flush=True)
            coordp = []
            peaks = []
            comet={}
            b={"comet":i, 'assaycomet':names[24:36]+"_"+str(i)}
            dict_var.update(b)
            src = tuple([x*subfactor for x in comet_rois[i][0]])
            dst = tuple([x*subfactor for x in comet_rois[i][1]])
            angle = math.atan2(dst[1]-src[1], dst[0]-src[0])
            length = math.sqrt((src[0] - dst[0])**2 + (src[1] - dst[1])**2)
            for s in np.arange(comet_rois[i][2][0] - 1,comet_rois[i][2][1],1):
                if s == comet_rois[i][2][0] - 1: #initial slice
                    #rotate the image to have a horizontal comet trajectory, rotate around origin of the comet so origin coordinates are unchanged
                    imrot = rotate(imstack[s], math.degrees(angle), center = (src[0],src[1]))
                    
                    # crop around the comet
                    crop = imrot[int(src[1]-2*deltay-1):int(src[1]+2*deltay), int(src[0]-(comet_length + deltax)):int(src[0]+length+(comet_length/2 + deltax))]
                    
                    #locate the user defined origin of the comet within the crop region:
                    src_cropx = int(comet_length + deltax)
                    src_cropy = int(2*deltay + 1)

                    #region where to seach for peakpixel
                    scan = crop[src_cropy - deltay:src_cropy + deltay, src_cropx - deltax:src_cropx + deltax]
                    peakpix = np.array(np.where(crop == np.max(scan)))
                    peakpix = np.array([peakpix[0][-1],peakpix[1][-1]]) #take the peak pixel further away from MT end in case many pixels have the same peak intensity
                    peaks.append(peakpix)
                    coordp.append(peakpix[1])
                    fullarea = crop[peakpix[0]-deltay:peakpix[0]+deltay, peakpix[1]- int(comet_length)-1 :peakpix[1]+int((comet_length)/2)]
                    #create an array of coordinates centered around 0
                    x_ip=np.linspace(((- int(((comet_length 
                                               -1
                                              )/2))) )* to_um , (int((comet_length )) ) * to_um,fullarea.shape[1])
                    x_ip=np.around(x_ip,decimals=4)
                    comet={"slice":s, "x_coord":x_ip}
                    d={}
                    for row in range(fullarea.shape[0]):
                        d[row]=fullarea[row]
                    comet.update(d)
                    if fullarea.shape==(subfactor * searchy * 2,comet_length + int(0.5 * comet_length) +1):
                        dict_var.update(comet)
                        df_comet=pd.DataFrame(dict_var)
                        list_df.append(df_comet) 
                    else:
                        print('fullareashape',fullarea.shape)
                else:#slices with comet after the first slice, search area is based on the peak pixel of the previous slice
                    
                    #rotate the image to have a horizontal comet trajectory, rotate around origin of the comet so origin coordinates are unchanged
                    imrot = rotate(imstack[s], math.degrees(angle), center = (src[0],src[1])) 
                    crop = imrot[int(src[1]-2*deltay-1):int(src[1]+2*deltay), int(src[0]-(comet_length + deltax)):int(src[0]+length+(comet_length/2 + deltax))]

                        
                    prec_peak = peaks[len(peaks)-1]
                    scan = crop[prec_peak[0] - deltay:prec_peak[0] + deltay, prec_peak[1] + 1:prec_peak[1] + 1 + deltax]
                    if ((scan.shape[0] == 0) &(scan.shape[1] == subfactor * searchy * 2)):
                        print('wrong')
                        peakpix = None
                        fullarea=np.zeros([0,comet_length + int(0.5 * comet_length) +1])
                        empty=empty+1

                        
                    elif ((scan.shape[0] == subfactor * searchy * 2) &(scan.shape[1] == 0)):
                        print('wrong2')
                        peakpix = None
                        fullarea=np.zeros([0,comet_length + int(0.5 * comet_length) +1])
                        empty=empty+1
                        
                        
                    else:
                        peakpix = np.array(np.where(crop == np.max(scan)))#.flatten()
                        peakpix = np.array([peakpix[0][-1],peakpix[1][-1]])
                        fullarea = crop[peakpix[0]-deltay:peakpix[0]+deltay, peakpix[1]- int(comet_length)-1 :peakpix[1]+int((comet_length)/2)]
                        peaks.append(peakpix)
                        coordp.append(peakpix[1])

                    x_ip=np.linspace(((- int(((comet_length 
                                               -1
                                              )/2))) )* to_um , (int((comet_length ) ) ) * to_um,fullarea.shape[1])#inverted because profile also inverted so that MT part on the right
                    x_ip=np.around(x_ip,decimals=4)
                
                    comet={"slice":s, "x_coord":x_ip}
                    d={}
                    for row in range(fullarea.shape[0]):
                        d[row]=fullarea[row]
                    comet.update(d)
                    if fullarea.shape==(subfactor * searchy * 2,comet_length + int(0.5 * comet_length) +1):
                        dict_var.update(comet)
                        df_comet=pd.DataFrame(dict_var)
                        list_df.append(df_comet)
                    #else:
                        #print('fullareashape',fullarea.shape)
            #if enough frames recorded, calculate growth velocity of comet            
            if len(np.diff(coordp)) == 0:
                average_vel = 0
            else:
                average_vel=((sum(np.diff(coordp))*(pixel_size/subfactor))/((len(np.diff(coordp)))*spf))*60
            #fit a linear regression to the positions of the comet for quality assesment
            slope, intercept, r_value, p_value, std_err = linregress(np.arange(0, (len(coordp))*spf, spf), coordp)
            dict_velocity={ 'assaycomet':names[24:36]+"_"+str(i), 'avg_velocity':average_vel, 'linear': r_value ** 2, 'empty': empty}
            
            empty=0
            df1velocity=pd.DataFrame.from_records([dict_velocity])
            list_velocity.append(df1velocity)
            
df_velocity=pd.concat(list_velocity)                
df=pd.concat(list_df)
df=pd.merge(df, df_velocity,on='assaycomet')

In [25]:
#Create super average of profiles for each species and velocity bin
df=df[df['avg_velocity']>0] #remove comet with a null growth velocity
df = df[df['linear']>0.9] #keep only comets with linear growth
df=df[df['empty']==0]
#add velocity bins to the dataframe
bins=np.arange(df['avg_velocity'].min(), df['avg_velocity'].max(), step=0.4)
bins=np.around(bins,decimals=1)
data=df
data['vel_binned'] = pd.cut(data['avg_velocity'],bins=bins)
rows = [x for x in list(range(0,24))]
data=data[['specie', 'assay', 'assaycomet', 'comet', 'vel_binned','x_coord','avg_velocity']+rows+['slice']]
data = data.drop(data[data.vel_binned.isnull()].index)
data['vel_binned']=data['vel_binned'].astype('str')

#select the velocities where more than 100 comets were recorded
data_filt=data.groupby(['vel_binned','assaycomet','slice']).first().reset_index()
data_filt=data_filt.groupby(['specie','vel_binned']).filter(lambda x: len(x) >= 100)
filtered_comets=data_filt['assaycomet'].unique().tolist()
data=data[data['assaycomet'].isin(filtered_comets)]

datag = data.groupby(['vel_binned','specie']).count().reset_index()
list3spe=[]
for i in datag['vel_binned'].unique().tolist():
    if len(datag[datag['vel_binned']==i]) == 3:
        list3spe.append(i)
data = data[data['vel_binned'].isin(list3spe)]
lst_profile = []
for s in data.specie.unique().tolist():
    dfa = data[data['specie']==s]
    for v in dfa.vel_binned.unique().tolist():
        #print(s, v)
        dfv = dfa[dfa['vel_binned']==v]
        dfv=dfv[['assaycomet','x_coord']+rows]
       # g = dfv.groupby(['assaycomet','x_coord']).mean()
        groups = dfv.groupby(["assaycomet",'x_coord'])
        mtmean=groups.aggregate(np.mean)
        #print(mtmean.shape)
        mtmean=mtmean.droplevel('x_coord')
        assays = mtmean.index.unique().tolist()
        mtmean=mtmean.reset_index()
        #print(mean)
        lst_com=[]
        
        for assay in assays:
            com=mtmean[mtmean['assaycomet']==assay]
            com=com.drop(['assaycomet'], axis=1)
            com=com.to_numpy()
            com=com[int(comet_length/4):,:]
            #print(com.shape)
            comnorm=normcomet(com)
            #print(np.where(comnorm == np.amax(comnorm)))
            lst_com.append(comnorm)
           # print(lst_com)
            
        supermean=np.mean(lst_com, axis=0)
        superstd = np.std(lst_com, axis = 0)
        supersem =  np.divide(superstd, np.sqrt(len(lst_com)))
        profile_supermean = supermean[:, 11:13].mean(axis=1)
        profile_std = superstd[:, 11:13].mean(axis=1)
        profile_sem =supersem[:, 11:13].mean(axis=1)
        profile_data={'specie':s,'vel_binned':v,'x_coord':x_ip[int(comet_length/4):]
                      , 'profile': profile_supermean[::-1], 'sem':profile_sem[::-1], 'std':profile_std[::-1]}
        
        dict_df = pd.DataFrame(profile_data)
        lst_profile.append(dict_df)
        imageio.imwrite((s + v + ' Average Cometsubfactor6_230107.tif'),supermean)
dp=pd.concat(lst_profile)
df_allvel=data.groupby(['specie', 'assaycomet','avg_velocity']).first().reset_index()
df_allvelbin=df_allvel.groupby(['specie', 'vel_binned']).agg({'avg_velocity': ['mean', 'std','count']}).reset_index()
df_allvelbin.columns=['specie','vbin','mean_vel','std_vel','count_vel']

In [27]:
#fit a Gaussian on the background part of the profiles and an exponential decay function on the microtubule part of the profile
import lmfit
from lmfit import Model, Parameter, report_fit
def normalized_data(data, data_err):
    normdata = (data-np.min(data))/(np.amax(data)-np.amin(data))
    normerr = (data_err)/(np.amax(data)-np.amin(data))
    return(normdata, normerr)
##################################
fitparamsdict=defaultdict(dict)
fitparams_lst = []
vel_binned_lst= np.unique(dp.vel_binned).tolist()
list_dfprof = []
list_dfcom = []
resultdict = {}
listres=[]
def decay(t, N, tau, I):
    return N*np.exp(-t/tau) + I


mod  = lmfit.models.GaussianModel() 

for vbin in vel_binned_lst:
    intensities=dp[dp['vel_binned']==vbin]
    intwide=intensities.pivot_table(index='x_coord',columns=['specie'])
    
    for s in np.unique(intensities.specie).tolist():
        
        dpi = intensities[intensities['specie']==s].dropna()
        #coord_plot = dpi['x_coord'].values
        data = dpi['profile'].values
        data_err = dpi['sem'].values
        data, data_err = normalized_data(data, data_err)

        data_exp= data[-(100-6):] #100 is the coordinate of max pixel when generating comet image

        coord_gauss = len(data)-(100-6)
        data_gauss = data[:coord_gauss]
        coord_plot=np.arange(- (len(data_gauss)), len(data_exp), 1)
        data_err = dpi['sem'].values
        lattice_intensity = data[-6:].mean()

        x_data_exp =(np.arange(0, data_exp.size))*pixel_size*1000/subfactor
        x_data_gauss =(np.arange(0, data_gauss.size))*pixel_size*1000/subfactor
        model = Model(decay)
        result_exp = model.fit(data_exp, t=x_data_exp, N=Parameter('N', value = data_exp[0], vary = False),I=Parameter('I', value = lattice_intensity, vary = False), tau=2)

        print(vbin, s, result_exp.fit_report())

        tau_f = result_exp.values['tau']
        std_tauf = result_exp.params['tau'].stderr
        pars_gauss = mod.guess(data_gauss, x=x_data_gauss)
        res_gauss = mod.fit(data_gauss, pars_gauss, x=x_data_gauss)
        exp_fit = decay(x_data_exp, data_exp[0], tau_f, lattice_intensity)
        gauss_fit = res_gauss.best_fit
        comb_fit = np.concatenate((gauss_fit, exp_fit), axis=0)

        globaldict={'specie': s,'velbinned':vbin,'coordnm':coord_plot*pixel_size/subfactor,'datainv':data, 'dataerrinv':data_err,  'fit': comb_fit }
        resultdict={'vbin':vbin,'specie':s, 'comet_length': tau_f, 'comet_length_SE': std_tauf}
        df_profiles = pd.DataFrame.from_dict(globaldict)
        list_dfprof.append(df_profiles)
        listres.append(resultdict)
df_profiles = pd.concat(list_dfprof)
df_res = pd.DataFrame(listres)

(1.4, 1.8] Xb [[Model]]
    Model(decay)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 9
    # data points      = 94
    # variables        = 1
    chi-square         = 0.00288210
    reduced chi-square = 3.0990e-05
    Akaike info crit   = -974.897817
    Bayesian info crit = -972.354522
[[Variables]]
    N:    0.6860813 (fixed)
    tau:  208.213019 +/- 1.00055996 (0.48%) (init = 2)
    I:    0.03156371 (fixed)
(1.4, 1.8] Xl [[Model]]
    Model(decay)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 9
    # data points      = 94
    # variables        = 1
    chi-square         = 0.00887387
    reduced chi-square = 9.5418e-05
    Akaike info crit   = -869.186277
    Bayesian info crit = -866.642982
[[Variables]]
    N:    0.6642536 (fixed)
    tau:  190.779674 +/- 1.73580478 (0.91%) (init = 2)
    I:    0.02633092 (fixed)
(1.4, 1.8] Xt [[Model]]
    Model(decay)
[[Fit Statistics]]
    # fitting method   = leastsq
    # functio

In [28]:
#plot the profiles and their fit
points = alt.Chart().mark_point(
    filled=True,
    size=10,
    #color='color=alt.Color('specie:N', scale=alt.Scale(scheme='pastel1'))'
).encode(
    x=alt.X('coordnm:Q'),
    y=alt.Y('datainv:Q', title = 'EB signal (normalized)'),
    color=alt.Color('specie:N', scale=scalesp2
                   )
)

# generate the error bars
errorbars = alt.Chart().mark_errorbar().encode(
    x="coordnm:Q",
    y=alt.Y("errmin:Q",title=None),
    y2="errmax:Q",
    color=alt.Color('specie:N', scale=scalesp2
                   )
)

convolved_profile =alt.Chart().mark_line(opacity=0.5
    #filled=True,
    #size=3,
    
).encode(
    x=alt.X('coordnm:Q', title = 'Distance (µm)'),
    y=alt.Y('fit:Q'),
    color=alt.Color('specie:N', scale=scalesp2
                   )
)

alt.layer(points, errorbars, convolved_profile, data = df_profiles).facet(column = alt.Column('velbinned', title='Growth velocity binned, µm/min') ).configure_axis(
                grid=False, 
                ticks=True, 
                labelPadding=0, 
                labelFontSize = 15, 
                labelFont = 'Helvetica Neue', 
                titleFontSize = 15
            ).configure_header(
                #labelOrient='bottom', 
                title=None, 
                labelFontSize=15, 
                labelFont='Helvetica Neue'
            ).configure_view(
                stroke='transparent'
            )

alt.FacetChart(...)

In [31]:
#table with results
dmat = pd.merge(df_allvelbin, df_res, on=['vbin','specie'])
dmat['decoration_s'] = (dmat['comet_length']/1000)/(dmat['mean_vel']/60)
dmat['ymin']=dmat['comet_length']-dmat['comet_length_SE']
dmat['ymax']=dmat['comet_length']-dmat['comet_length_SE']
dmat['vbin']=dmat['vbin'].astype(str)
dmat

,specie,vbin,mean_vel,std_vel,count_vel,comet_length,comet_length_SE,decoration_s,ymin,ymax
0,Xb,"(1.4, 1.8]",1.647199,0.112345,52,208.213019,1.000560,7.584256,207.212459,207.212459
1,Xb,"(1.8, 2.2]",2.002164,0.114927,103,224.342102,1.267623,6.722988,223.074479,223.074479
2,Xb,"(2.2, 2.6]",2.382824,0.104058,67,235.149325,1.828282,5.921108,233.321043,233.321043
3,Xl,"(1.4, 1.8]",1.635789,0.127159,29,190.779674,1.735805,6.997713,189.043869,189.043869
4,Xl,"(1.8, 2.2]",2.018002,0.121581,120,185.362202,1.424286,5.511260,183.937916,183.937916
5,Xl,"(2.2, 2.6]",2.359867,0.106645,52,213.667478,1.886319,5.432529,211.781158,211.781158
6,Xt,"(1.4, 1.8]",1.616814,0.104010,103,181.511345,1.176849,6.735890,180.334496,180.334496
7,Xt,"(1.8, 2.2]",2.012222,0.119615,103,201.381323,1.324230,6.004743,200.057093,200.057093
8,Xt,"(2.2, 2.6]",2.384236,0.109160,41,194.215197,1.803596,4.887482,192.411601,192.411601


In [30]:
#plot the results
chart=alt.Chart(dmat).mark_circle().encode(
    x=alt.X('mean_vel', title = 'Mean growth rate (µm/min)',scale=alt.Scale(zero=False)),
    y=alt.Y('comet_length', title = 'Mean comet length (nm)'),
    color=alt.Color('specie', scale=scalesp2)
).properties(
    width=400,
    height=300
)
errorbars = alt.Chart().mark_errorbar().encode(
        x="mean_vel:Q",
        y=alt.Y("ymin:Q", title = 'Mean comet length (nm)'),
        y2="ymax:Q",
       color = alt.Color('specie:Q',scale=scalesp2)
    )

regression=chart.transform_regression('mean_vel', 'comet_length', groupby = ['specie']).mark_line()
alt.layer(chart,
          errorbars,
          regression, data=dmat.reset_index()
        
        ).configure_header(
            labelFont='Helvetica Neue',
            labelFontStyle='bold italic',
            labelFontSize=20,
            title=None
).configure_axis(grid=False, 
             ticks=True, labelPadding=0, labelFontSize = 20, labelFont = 'Helvetica Neue', titleFontSize = 20, titleFont='Helvetica Neue')

alt.LayerChart(...)